In [161]:
import tensorflow as tf  # type: ignore
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input  # type: ignore
import numpy as np
import pandas as pd
from torchdata.datapipes.iter import IterDataPipe  # type: ignore
import logging
import time
import torch
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle 
from torch.utils.data import DataLoader  # TODO: upgrade to DataLoader2
from retrying import retry
import urllib.request
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [62]:
print(tf.__version__)

2.13.1


!aws s3 sync s3://sparp-dev/shilinz/image_embedding/models/ResNet50-allGLs-TripletLoss-CosineDist-DenseReLU1024-epoch5-v2/ ResNet50-allGLs-TripletLoss-CosineDist-DenseReLU1024-epoch5-v2

In [6]:
!ls ResNet50-allGLs-TripletLoss-CosineDist-DenseReLU1024-epoch5-v2/

keras_metadata.pb  saved_model.pb  variables


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
def process_image(img: np.ndarray, dim: int) -> np.ndarray:
    import cv2  # type: ignore

    img = cv2.imdecode(img, cv2.IMREAD_COLOR)
    if img is None:
        return img
    img = cv2.resize(img, (int(dim), int(dim)), interpolation=cv2.INTER_NEAREST)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [7]:
model_dir ='ResNet50-allGLs-TripletLoss-CosineDist-DenseReLU1024-epoch5-v2'
model = tf.keras.models.load_model(model_dir)

In [140]:
def _fetch_image_retryable_exceptions(exception: Exception) -> bool:
    # TODO: refine the retryable exceptions
    if "failure in name resolution" in str(exception).lower():
        logger.warning("failure in name resolution, retrying")
        return True
    return False


@retry(
    stop_max_attempt_number=20,
    retry_on_exception=_fetch_image_retryable_exceptions,
    wait_exponential_max=600_000,
)
def fetch_image(image_id: str) -> np.ndarray:
    url = f"https://m.media-amazon.com/images/I/{image_id}.jpg"
    with urllib.request.urlopen(url) as request:
        img_array = np.asarray(bytearray(request.read()), dtype=np.int8)
    return img_array

def fetch_pin_image(img_sig: str) -> np.ndarray:
    if len(img_sig) < 6:
        return ''
    first2 = img_sig[:2] + '/'
    second2 = img_sig[2:4] + '/'
    third2 = img_sig[4:6] + '/'
    pin_id = first2 + second2 + third2
    output_url = 'https://i.pinimg.com/564x/'+ pin_id + img_sig+'.jpg'
    with urllib.request.urlopen(output_url) as request:
        img_array = np.asarray(bytearray(request.read()), dtype=np.int8)
    return img_array

In [27]:
ASIN = 'asin'
IMAGE_ID = 'winning_image_physical_id'
IMAGE = 'asin_image'
MK_ID = "marketplace_id"

In [128]:
class LocalParquetFileLineReaderAmznImageEmbedDataPipe(IterDataPipe):
    def __init__(self, df, IMAGE_ID):
        self.df = df
        self.image_id = IMAGE_ID

    def __iter__(self):
#         pa_dataset = pq.ParquetDataset([self.file_path], filesystem=_find_filesystem(self.file_path))
#         df = pa_dataset.read(columns=["asin", "image_id", "marketplace_id"]).to_pandas()
        asins = self.df[self.image_id].to_numpy()
        image_ids = self.df[self.image_id].to_numpy()
        #images = self.df[IMAGE].to_numpy()
        print('start')
        for index in range(0, self.df.shape[0]): 
            asin = asins[index]
            image_id = image_ids[index]
            #img = images[index]
            try:
                img = fetch_image(image_id)
                img = process_image(img, 224)
                if img is None:
                    continue
            except Exception as e:
                print(f"Error when processing: {asin}", e)
                continue
            yield {"image_id": asin, "image": img}

In [129]:
class LocalParquetFileLineReaderPinImageEmbedDataPipe(IterDataPipe):
    def __init__(self, df, IMAGE_ID):
        self.df = df
        self.image_id = IMAGE_ID

    def __iter__(self):
#         pa_dataset = pq.ParquetDataset([self.file_path], filesystem=_find_filesystem(self.file_path))
#         df = pa_dataset.read(columns=["asin", "image_id", "marketplace_id"]).to_pandas()
        asins = self.df[self.image_id].to_numpy()
        image_ids = self.df[self.image_id].to_numpy()
        #images = self.df[IMAGE].to_numpy()
        print('start')
        for index in range(0, self.df.shape[0]): 
            asin = asins[index]
            image_id = image_ids[index]
            #img = images[index]
            try:
                img = fetch_pin_image(image_id)
                img = process_image(img, 224)
                if img is None:
                    continue
            except Exception as e:
                print(f"Error when processing: {asin}", e)
                continue
            yield {"image_id": asin, "image": img}

In [17]:
with open('/home/ec2-user/SageMaker/pinterest/data/RP_label_sample_2images_clipEmb.p', 'rb') as fp:
    input_data = pickle.load(fp)

In [136]:
def batchinfer_fn(
    model,
    input_data,
    item_id="asin",
    image_id = 'physical_id',
    #config: Dict[str, Any],
) -> pd.DataFrame:
    """

    :param model:
    :param input_file:
    :param config:
    :return:
    """

    from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input  # type: ignore

    start_time = time.time()
    if item_id in ['asin','ASIN']:
        dataset = LocalParquetFileLineReaderAmznImageEmbedDataPipe(input_data,image_id )
    else:
        dataset = LocalParquetFileLineReaderPinImageEmbedDataPipe(input_data, image_id)
    embeddings_list = []
    asins_list = []
    cnt = 0
    batch_size = 128
    # TODO: move DataLoader as a generic implementation in torchdata_batchinfer
    for batch in DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=0,  # disable multiprocess since it's not a real datapipe implementation
    ):
        cnt += 1
        asins = np.array(batch['image_id']).astype(str)
        asins_list.append(asins)
        images = batch["image"].numpy()
        preprocessed_images = preprocess_input(images)
        embeddings = model.predict(preprocessed_images)
        embeddings_list.append(embeddings)
        if cnt % 100 == 0:
            logger.info(f"Elapsed time: {time.time() - start_time} seconds for {cnt * batch_size} records")
    merged_asins = np.concatenate(asins_list, axis=0)
    merged_embeddings = np.concatenate(embeddings_list, axis=0)
    df_asins = pd.DataFrame(merged_asins, columns=[item_id])
    embed_cols = [f"embed_{x}" for x in range(0, 1024)]
    df_flat_embeds = pd.DataFrame(merged_embeddings, columns=embed_cols)
    df = pd.concat([df_asins, df_flat_embeds], axis=1)
    df["embed"] = df[embed_cols].apply(lambda x: np.array(x), axis=1)
    df.drop(columns=embed_cols, inplace=True)
    return df

In [146]:
asin_emb = batchinfer_fn(model, input_data.drop_duplicates('asin'), 'asin','winning_image_physical_id' )

start
4/4 [==============================] - 0s 102ms/step


In [150]:
asin_emb.shape, asin_emb.drop_duplicates('asin').head(2)

((1762, 2),
           asin  \
 0  419KF50+zuL   
 1  414JR9deANL   
 
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 asin_resnet50_emb  
 0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.32235116, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3172164, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [143]:
pin_emb = batchinfer_fn(model, input_data[~input_data['img_sig'].isna()].drop_duplicates('img_sig'), 'img_sig','img_sig' )

start
1/1 [==============================] - 0s 37ms/step
